In [ ]:
import time
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

if torch.cuda.is_available():
  torch.backends.cudnn.deterministic = True

In [ ]:
device = torch.device ("cuda:3" if torch.cuda.is_available() else "cpu")

In [ ]:
#Hyperparameters
#learning_rate = 0.05
#num_epochs = 10
batch_size = 128

# Architecture
#num_classes = 10

In [ ]:
#!wget --no-check-certificate "https://github.com/TimG27/demo_cnn/blob/main/tr_dataset.zip"

In [ ]:

#import zipfile

#import tarfile
#tar = tarfile.open('tr_dataset.zip', 'r:gz')
#tar.extractall()
#tar.close()

#zip_ref = zipfile.ZipFile('tr_dataset.zip', 'r') #Opens the zip file in read mode
#zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
#zip_ref.close()

In [64]:
#!wget --no-check-certificate \
 #   "https://github.com/TimG27/demo_cnn/archive/refs/heads/main.zip" \
  #  -O "/ptmp/tr_dataset.zip"

#import zipfile

#zip_ref = zipfile.ZipFile('/ptmp/tr_dataset.zip', 'r') #Opens the zip file in read mode
#zip_ref.extractall('/ptmp') #Extracts the files into the /tmp folder
#zip_ref.close()

In [65]:
!wget -cq https://github.com/TimG27/demo_cnn/raw/main/tr_dataset.zip

In [66]:
import zipfile

zip_ref = zipfile.ZipFile('tr_dataset.zip', 'r') 
zip_ref.extractall() 
zip_ref.close()

In [102]:
import torchvision

transformer = transforms.Compose([
                                  transforms.Resize((256,256)),
                                  transforms.ToTensor()])

In [104]:

train_path = '/content/train'
test_path = '/content/test'

batch_size = 16

train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path, transform = transformer), 
                          batch_size=batch_size, 
                          shuffle=True)
test_loader = DataLoader (torchvision.datasets.ImageFolder(test_path, transform = transformer),
                          batch_size = batch_size, shuffle = True)

In [105]:
import pathlib

root = pathlib.Path (train_path)
classes = sorted ([j.name.split('/')[-1] for j in root.iterdir()])

In [106]:
print(classes)

['retroflex-rectum']


In [107]:
for images, labels in test_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([2, 3, 256, 256])
Image label dimensions: torch.Size([2])


In [112]:
import torch.nn as nn

class FireModule (nn.Module):

    def __init__(self, inplanes, squeeze_planes, expand1x1_planes, expand3x3_planes):
        super(FireModule, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(x))
        ], 1)

In [192]:
num_classes = 2

class FireBlock(nn.Module):

    def __init__(self, num_classes):
        super(FireBlock, self).__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
                nn.Conv2d(3, 96, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                
                FireModule(96, 16, 64, 64),
                FireModule(128, 16, 64, 64),

                nn.Conv2d(128, self.num_classes, kernel_size=7),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
        )

    def forward(self, x):
        x = self.features(x)
        #x = self.classifier(x)
        return torch.flatten(x, 1)
        #return x;


'''
                FireModule(128, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                FireModule(256, 32, 128, 128),
                FireModule(256, 48, 192, 192),
                FireModule(384, 48, 192, 192),
                FireModule(384, 64, 256, 256),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                FireModule(512, 64, 256, 256),
            )

        # Final convolution is initialized differently from the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal_(m.weight, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
'''


'\n                FireModule(128, 32, 128, 128),\n                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),\n                FireModule(256, 32, 128, 128),\n                FireModule(256, 48, 192, 192),\n                FireModule(384, 48, 192, 192),\n                FireModule(384, 64, 256, 256),\n                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),\n                FireModule(512, 64, 256, 256),\n            )\n\n        # Final convolution is initialized differently from the rest\n        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)\n        self.classifier = nn.Sequential(\n            nn.Dropout(p=0.5),\n            final_conv,\n            nn.ReLU(inplace=True),\n            nn.AdaptiveAvgPool2d((1, 1))\n        )\n\n        for m in self.modules():\n            if isinstance(m, nn.Conv2d):\n                if m is final_conv:\n                    init.normal_(m.weight, mean=0.0, std=0.01)\n                else:\n                  

In [193]:
model = FireBlock (num_classes = 1).to(device)

In [194]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [195]:
num_epochs = 10

In [196]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [197]:
print(train_count,test_count)

6 2


In [198]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch: 0 Train Loss: tensor(6.6658) Train Accuracy: 0.0 Test Accuracy: 0.5
Epoch: 1 Train Loss: tensor(6.6579) Train Accuracy: 0.3333333333333333 Test Accuracy: 1.0
Epoch: 2 Train Loss: tensor(6.6293) Train Accuracy: 0.8333333333333334 Test Accuracy: 1.0
Epoch: 3 Train Loss: tensor(6.5806) Train Accuracy: 0.8333333333333334 Test Accuracy: 1.0
Epoch: 4 Train Loss: tensor(6.5043) Train Accuracy: 0.8333333333333334 Test Accuracy: 1.0
Epoch: 5 Train Loss: tensor(6.4000) Train Accuracy: 0.8333333333333334 Test Accuracy: 1.0
Epoch: 6 Train Loss: tensor(6.2723) Train Accuracy: 0.8333333333333334 Test Accuracy: 1.0
Epoch: 7 Train Loss: tensor(6.1037) Train Accuracy: 0.8333333333333334 Test Accuracy: 1.0
Epoch: 8 Train Loss: tensor(5.8942) Train Accuracy: 1.0 Test Accuracy: 1.0
Epoch: 9 Train Loss: tensor(5.6467) Train Accuracy: 1.0 Test Accuracy: 1.0
